In [1]:
from transformers import pipeline, Conversation, AutoModelForCausalLM, AutoTokenizer, AutoModelForSeq2SeqLM
import torch
model_name = "NousResearch/Hermes-2-Pro-Llama-3-8B"

print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(torch.cuda.current_device()))

True
0
NVIDIA GeForce RTX 2070 SUPER


In [2]:


tokenizer = AutoTokenizer.from_pretrained(model_name)
#model = AutoModelForCausalLM.from_pretrained(model_name)

# using Seq2Seq pipeline model=model. using Causal pipeline model=model breaks. Causal pipline model=model_name works fine but
# just for potential model customization in future, like down here with torch_dtype, using model=model better so need to use Seq2Seq
# torch_dtype=torch.float16, load_in_8bit=True REDUCES TIME FROM 12S TO 3S! confirmed via testing.
model = AutoModelForCausalLM.from_pretrained(model_name,
        torch_dtype=torch.float16, load_in_8bit=True)


#generator = pipeline("conversational", model=model, tokenizer=tokenizer)   # takes upwards of 1 min, immeseured.
#generator = pipeline("conversational", model=model_name)     # 12 sec
generator = pipeline("conversational", model=model, tokenizer=tokenizer)   # 15sec

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
convo = Conversation("Hi, what's your name?")


In [4]:
response = generator(convo) 

c:\Users\hbros\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\models\llama\modeling_llama.py:649: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [5]:
print(response)

Conversation id: 8bead4b6-068b-4f00-97af-5d35145f5a1a
user: Hi, what's your name?
assistant: Hello! My name is Open Assistant. How can I assist you today?



Now to add a way for user inputs

In [8]:
import gradio as gr

In [10]:
inputs = []
responses = []
# inputs and responses could be combined to form a 'memory' of sorts
# how to implement backstory/key memories? need to research

def convo_bot(input, response):
    convo = Conversation(text=input, past_user_inputs=inputs, generated_responses=responses)
    convo = generator(convo)
    return convo.generated_responses[-1]

demo = gr.ChatInterface(convo_bot, title="Vanilla Chatbot", description="...")
demo.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
